In [1]:
#!pip install geopy
#!pip install folium
import pandas as pd
import statsmodels as sm
import matplotlib.pyplot as plt
import json
import folium
from geopy.geocoders import Nominatim
from folium.plugins import HeatMap

In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [2]:
df = pd.read_csv("noislamophobia-dataset-50k.csv",low_memory=False)

In [3]:
place_df = df[df["place"].notna()].astype('str')
place_df = place_df["place"]
print(place_df)

94       {"id":"1d68da80ca90416d","url":"https://api.tw...
206      {"id":"53cef5332ac9d7d0","url":"https://api.tw...
638      {"id":"f632697d33274211","url":"https://api.tw...
642      {"id":"5635c19c2b5078d1","url":"https://api.tw...
997      {"id":"a81f9ed24c15d6af","url":"https://api.tw...
                               ...                        
48145    {"id":"74effb34539b2d64","url":"https://api.tw...
48146    {"id":"74effb34539b2d64","url":"https://api.tw...
48343    {"id":"01693a337224bad0","url":"https://api.tw...
49347    {"id":"07d9e3eb0a486002","url":"https://api.tw...
50162    {"id":"019f39d424818f2b","url":"https://api.tw...
Name: place, Length: 251, dtype: object


In [4]:
places = place_df.apply(json.loads).apply(pd.Series)
places['full_name'] = places['full_name'].str.replace(', USA', '')
places['location'] = places['full_name'] + ", " + places['country']
places.head()

,id,url,place_type,name,full_name,country_code,country,contained_within,bounding_box,attributes,location
94,1d68da80ca90416d,https://api.twitter.com/1.1/geo/id/1d68da80ca9...,city,Kawartha Lakes,"Kawartha Lakes, Ontario",CA,Canada,[],"{'type': 'Polygon', 'coordinates': [[[-79.2086...",{},"Kawartha Lakes, Ontario, Canada"
206,53cef5332ac9d7d0,https://api.twitter.com/1.1/geo/id/53cef5332ac...,city,Waunfawr,"Waunfawr, Wales",GB,United Kingdom,[],"{'type': 'Polygon', 'coordinates': [[[-4.21700...",{},"Waunfawr, Wales, United Kingdom"
638,f632697d33274211,https://api.twitter.com/1.1/geo/id/f632697d332...,city,Mount Airy,"Mount Airy, MD",US,United States,[],"{'type': 'Polygon', 'coordinates': [[[-77.1957...",{},"Mount Airy, MD, United States"
642,5635c19c2b5078d1,https://api.twitter.com/1.1/geo/id/5635c19c2b5...,admin,Virginia,Virginia,US,United States,[],"{'type': 'Polygon', 'coordinates': [[[-83.6752...",{},"Virginia, United States"
997,a81f9ed24c15d6af,https://api.twitter.com/1.1/geo/id/a81f9ed24c1...,city,Grand Rapids,"Grand Rapids, MI",US,United States,[],"{'type': 'Polygon', 'coordinates': [[[-85.7514...",{},"Grand Rapids, MI, United States"


In [5]:
locator = Nominatim(user_agent='myGeocoder')
m = folium.Map(location=[20,0], zoom_start=2)
data = []
for index, row in places.iterrows():
    location = locator.geocode(row['location'])
    if (location == None):
        location = locator.geocode(row['country'])
        
    data.append([location.latitude, location.longitude, 1])
    
    folium.Circle(
        radius=400,
        location=[location.latitude, location.longitude],
        popup=row["name"],
        color="crimson",
        fill=False,
    ).add_to(m)
places_copy = pd.DataFrame(data, columns=['latitude', 'longitude', 'count'])
m.save('50kdotmap.html')
m

In [9]:
n = folium.Map(location=[20,0], zoom_start=2)
HeatMap(data=places_copy[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=10, max_zoom=10).add_to(n)
n.save('50kheatmap.html')
n